# Variance magnitude analysis

## Defined output variance
Based on the linear error propagation, we can explore a suitable uncertainty of the input by defining an acceptable output variance. 

Let $I \in \mathbb{N}^{r \times v}$ be a sinogram and the function $f: \mathbb{N}^{r \times v} \rightarrow \mathbb{N}^{n \times n}$ is the inverse radon transorm. Further be $F_x$ the Jacobian of $f$:
\begin{align}
    F_x &= \left[ \nabla f^T \right]^T &= \left[\begin{array}{c}
        f_1 \\
        \vdots \\
        f_{nn}
    \end{array}\right] \left[\begin{array}{ccc} \frac{\partial}{\partial I_1} & \dots & \frac{\partial}{\partial I_{rn}} \end{array}\right] &= \left[\begin{array}{ccc}
        \frac{\partial f_1}{\partial I_1} & \dots & \frac{\partial f_1}{\partial I_{rv}} \\
        \vdots & \ddots & \vdots \\
        \frac{\partial f_{nn}}{\partial I_1} & \dots & \frac{\partial f_{nn}}{\partial I_{rv}}
    \end{array}\right]
\end{align}

We now assume that $I$~$\mathcal{N}(\mu, \Sigma)$ is a realization of a multivariate normal distribution. The covariance matrix $\Sigma$ can be modelled in a way, that the correlation factor $\rho(i, j)$ is higher for columns $j$ and stays constant for rows $i$. We use exponential decay for the columns:
$$
    \rho(i) = exp( - \gamma i)
$$

The propagation of the uncertanty $\Sigma$ can be computed as followed:
$$
    \Sigma_f = F_x \Sigma F_x^T
$$

Now, we define an acceptable $\Sigma_f \leq \varepsilon$ and identify $\Sigma$, which fullfills this condition by iterative exploration.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from Comparision import utils

In [2]:
df_dx = np.load('../Data/iradon/df_dx_ramp.npy')

@widgets.interact(input_range=(10,1500))
def show_variance(input_range = 1100):
    width = 5
    
    var = np.ones((64,64)) * input_range
    Sigma_in = utils.build_covariance_y(var, function=utils.exponential, width=width)

    Sigma = df_dx @ Sigma_in @ np.transpose(df_dx)

    variance = np.reshape(np.abs(np.diag(Sigma)), (64,64))

    plt.figure(figsize=(18,6))
    plt.imshow(variance)
    plt.colorbar()
    plt.show()

interactive(children=(IntSlider(value=1100, description='input_range', max=1500, min=10), Output()), _dom_clas…

## Defined input variance
We can identify a rough uncertainty estimation in the sinogram by error propagation of the forward radon transform.

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from skimage import transform

In [4]:
kidney = np.loadtxt('../Data/kidney.csv', delimiter=',', skiprows=1, usecols=(0), dtype=np.float32).reshape((512,512))

#enforce to be zero on the edges
kidney[:,:10] = 50
kidney[:,-10:] = 50
kidney[:10,:] = 50
kidney[-10:,:] = 50

kidney = kidney[::8,::8]
img_shape = kidney.shape[0]

In [7]:
img = tf.Variable(kidney)
img = tf.reshape(img,(1,64,64,1))
with tf.GradientTape() as tape:
    tape.watch(img)
    theta = 20.0 * np.pi/180
    
    #rotated = tfa.image.rotate(img, theta, interpolation='bilinear', fill_mode='constant', fill_value=0)
    rotated = tfa.image.transform(img, [np.cos(theta), -np.sin(theta), 0,
                                        np.sin(theta), np.cos(theta), 0,
                                        0, 0])
        
    sinogram = tf.reduce_sum(rotated, 0)
    
    #reconstructed = transform.iradon(sinogram.numpy().T, np.arange(0,180))
    
    n = np.prod(img.shape)
    #df_dx = tape.gradient(rotated, img)
jacobian = tape.jacobian(rotated, img)
    #print(jacobian)
    #jacobian = tf.reshape(jacobian, (n,n))

plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
plt.imshow( tf.reshape(rotated, (64,64)) )
plt.colorbar()

plt.subplot(1,2,2)
#plt.imshow(df_dx)
plt.colorbar()

plt.show()

plt.figure(figsize=(18,6))
#plt.imshow(jacobian)
plt.show()

plt.figure(figsize=(18,6))
#plt.imshow( tf.reshape(tf.linalg.tensor_diag_part(jacobian), kidney.shape) )
plt.show()

NotImplementedError: in user code:

    /home/vik/Programme/anaconda3/envs/testing/lib/python3.9/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:190 f  *
        parallel_iterations=parallel_iterations)
    /home/vik/Programme/anaconda3/envs/testing/lib/python3.9/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:248 _pfor_impl  **
        loop_fn_outputs = loop_fn(loop_var)
    /home/vik/Programme/anaconda3/envs/testing/lib/python3.9/site-packages/tensorflow/python/eager/backprop.py:1178 loop_fn
        return self.gradient(y, flat_sources,
    /home/vik/Programme/anaconda3/envs/testing/lib/python3.9/site-packages/tensorflow/python/eager/backprop.py:1080 gradient
        flat_grad = imperative_grad.imperative_grad(
    /home/vik/Programme/anaconda3/envs/testing/lib/python3.9/site-packages/tensorflow/python/eager/imperative_grad.py:71 imperative_grad
        return pywrap_tfe.TFE_Py_TapeGradient(
    /home/vik/Programme/anaconda3/envs/testing/lib/